> To live! like a tree alone and free,
> and like a forest in solidarity...
> Nazim Hikmet

<center> <h1> Random Forests (Yes! No Forest Image) </h1> </center>



__Objectives__

- Introduction of 'bagging' procedure.

- Identifying the need for bootstrapping for decision trees.

- Comparing Random forests and bagging methods

- Evaluating a model by random forest model

# Review: Bootstrapping


<img src= "img/bootstrap1.png" style="height:250px">


# Bagging


Let's us one more time recall that if $Z_{1}, \cdots, Z_{n}$ are independent observations with variance $\sigma^{2}$ then the variance of the mean $\bar{Z}$ is given by $\frac{\sigma^{2}}{n}$. 

__How is this relevant now?__



We will use this idea calculate $$ \hat{f}^{1}(x), \cdots, \hat{f}^{B}(x)$$ where each $\hat{f}^{i}$ represents a decision tree fitted to the bootstrapped data.

Then we will make a prediction by: 

$$ \hat{f}_{\text{avg}}(x) = \frac{1}{B}\sum_{b=1}^{B} \hat{f}^{b}(x)$$

Note that this is for regression and for the classification we can get majority vote.

[sklearn averages over probabilities not majority vote](https://scikit-learn.org/stable/modules/ensemble.html#forest)


## Random Forests

__Problem__ We still have some problem with this approach and random forests will address this problem. Can you see the issue?

- If we have a strong predictor then this will dominate in each tree.

Hint: Correlated trees

## Sklearn for RandomForests

In [1]:
import pandas as pd

In [2]:
## you can download the data from -- https://www.kaggle.com/ishaanv/ISLR-Auto#Heart.csv

## or http://faculty.marshall.usc.edu/gareth-james/ISL/data.html
heart = pd.read_csv('data/Heart.csv', index_col = 0)
heart.head()
print(heart.shape)

(303, 14)


In [3]:
heart.dropna(axis= 0, how= 'any', inplace = True)
y = heart.AHD
heart.drop(columns= 'AHD', inplace = True)

In [4]:
heart.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 297 entries, 1 to 302
Data columns (total 13 columns):
Age          297 non-null int64
Sex          297 non-null int64
ChestPain    297 non-null object
RestBP       297 non-null int64
Chol         297 non-null int64
Fbs          297 non-null int64
RestECG      297 non-null int64
MaxHR        297 non-null int64
ExAng        297 non-null int64
Oldpeak      297 non-null float64
Slope        297 non-null int64
Ca           297 non-null float64
Thal         297 non-null object
dtypes: float64(2), int64(9), object(2)
memory usage: 32.5+ KB


In [5]:
display(heart.Thal.value_counts())
display(heart.ChestPain.value_counts())

normal        164
reversable    115
fixed          18
Name: Thal, dtype: int64

asymptomatic    142
nonanginal       83
nontypical       49
typical          23
Name: ChestPain, dtype: int64

In [6]:
from sklearn.model_selection import train_test_split 
X_train, X_test,y_train, y_test = train_test_split(heart, y, test_size= 0.20, stratify = y)

In [7]:
from sklearn.preprocessing import OneHotEncoder

In [13]:
categorical_variables = X_train.select_dtypes(include=['object']).columns
numerical_variables = X_train.select_dtypes(include = ['int64', 'float64']).columns
categorical_variables, numerical_variables

(Index(['ChestPain', 'Thal'], dtype='object'),
 Index(['Age', 'Sex', 'RestBP', 'Chol', 'Fbs', 'RestECG', 'MaxHR', 'ExAng',
        'Oldpeak', 'Slope', 'Ca'],
       dtype='object'))

In [9]:
import numpy as np

In [10]:
ohe = OneHotEncoder(drop = 'first')
X_categ = ohe.fit_transform(X_train[categorical_variables]).toarray()
X_num = X_train[numerical_variables].values
Xtrain = np.concatenate((X_categ, X_num), axis = -1,)
Xtrain.shape

(237, 16)

In [14]:
## now we should transform the test data 
## to be able to use it for the prediction

X_test_categ = ohe.transform(X_test[categorical_variables]).toarray()
X_test_num = X_test[numerical_variables].values
Xtest = np.concatenate((X_test_categ, X_test_num), axis = -1,)
Xtest.shape

(60, 16)

In [15]:
from sklearn.ensemble import RandomForestClassifier

In [97]:
clf = RandomForestClassifier(n_estimators= 100, 
                             criterion= 'gini', 
                             max_features= 'auto',
                             oob_score= True)

In [98]:
clf.fit(Xtrain, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=True, random_state=None,
                       verbose=0, warm_start=False)

In [99]:
clf.oob_score_

0.8143459915611815

In [118]:
clf_2 = RandomForestClassifier(n_estimators=31,
                             min_samples_split=5,
                             min_samples_leaf=4,
                             max_features='sqrt',
                             max_depth=3,
                             bootstrap=True,
                             oob_score=True)
clf_2.fit(Xtrain, y_train)
clf_2.oob_score_

0.8059071729957806

In [134]:
clf_3 = RandomForestClassifier(n_estimators=100,
                             criterion= 'gini', 
                             max_features='auto',
                             max_depth=4,
                             max_leaf_nodes=4,
                             oob_score=True)
clf_3.fit(Xtrain, y_train)
clf_3.oob_score_

0.8227848101265823

In [130]:
clf_4 = RandomForestClassifier(n_estimators=100,
                             criterion= 'gini', 
                             max_features='auto',
                             max_depth=None,
                             min_samples_split=2,
                             oob_score=True)
clf_4.fit(Xtrain, y_train)
clf_4.oob_score_

0.8059071729957806

In [135]:
clf_3.feature_importances_

array([0.01719089, 0.        , 0.        , 0.18844954, 0.16024387,
       0.02744644, 0.01184903, 0.01029863, 0.0084941 , 0.        ,
       0.00127021, 0.11527245, 0.13319266, 0.08197206, 0.09262483,
       0.15169529])

In [132]:
clf_4.feature_importances_

array([0.03494235, 0.01234375, 0.01177521, 0.07707454, 0.09127012,
       0.07730115, 0.03088459, 0.07983415, 0.0769276 , 0.00807003,
       0.01707479, 0.12264092, 0.08750433, 0.09342493, 0.06691495,
       0.1120166 ])

__Your Turn__

- Use 5 fold cross_validation to fit random forest classifier we created above.
- Don't forget to return training scores and trained estimators.

In [17]:
from sklearn.model_selection import cross_validate

In [53]:
# %load -r 1-6 supplement.py
cv = cross_validate(clf,
                    Xtrain,
                    y_train,
                    return_train_score=True,
                    return_estimator=True,
                    cv=5)

__Your Turn__

- What is the type of validator above?

- Check test vs train(validation) scores.

- Print "mean +/- std" for both train and test scores

- Also print oob_scores and compare them with cross_validation scores

In [54]:
# %load -r 9-17 supplement.py
mean_test = np.mean(cv['test_score'])
std_test = np.std(cv['test_score'])

mean_train = np.mean(cv['train_score'])
std_train = np.std(cv['train_score'])


print("Train score: %.3f +/- %.3f"%(mean_train, std_train))
print("Test score: %.3f +/- %.3f"%(mean_test, std_test))

Train score: 0.853 +/- 0.017
Test score: 0.806 +/- 0.073


__Your Turn__

- Note that we have over-fitting problem. 

- Let's try to reduce over-fitting

In [63]:
clf = RandomForestClassifier(n_estimators= 100, 
                             criterion= 'gini', 
                             max_features='auto',
                             oob_score= True,
                             max_depth=4,
                             max_leaf_nodes=4
                            )
cv = cross_validate(clf,
                    Xtrain,
                    y_train,
                    return_train_score=True,
                    return_estimator=True,
                    cv=5)
mean_test = np.mean(cv['test_score'])
std_test = np.std(cv['test_score'])

mean_train = np.mean(cv['train_score'])
std_train = np.std(cv['train_score'])


print("Train score: %.3f +/- %.3f"%(mean_train, std_train))
print("Test score: %.3f +/- %.3f"%(mean_test, std_test))

Train score: 0.854 +/- 0.018
Test score: 0.823 +/- 0.051


In [65]:
# %load -r 20-27 supplement.py
clf = RandomForestClassifier(n_estimators= 100,
                             criterion= 'gini',
                             max_features= 'auto', max_depth= 2,
                             oob_score= True)

clf.fit(Xtrain, y_train);
clf.score(Xtrain, y_train)
clf.oob_score_

0.8143459915611815

### Extra: Pipelines?

In [66]:
## There is an "easier" way to do this
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier

In [83]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(drop='first'))])

preprocessor = ColumnTransformer(
        transformers=[
        ('num', numeric_transformer, numerical_variables),
        ('cat', categorical_transformer, categorical_variables)],)

rf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestClassifier(n_estimators =100,
                                                            oob_score = True,
                                                            criterion= 'gini', 
                                                            max_features='auto',
                                                            max_depth=4,
                                                            max_leaf_nodes=4))])

In [84]:
pipe_validator = cross_validate(rf, 
                    X_train, 
                    y_train, 
                    return_train_score= True, 
                    return_estimator= True,
                    cv = 5)

In [85]:
## train scores
print(pipe_validator['train_score'])
## validation scores
print(pipe_validator['test_score'])
## oob scores
print([est['classifier'].oob_score_ for est in pipe_validator['estimator']])
## let's pick one of the estimator for further investigation

est = pipe_validator['estimator'][0]

[0.84656085 0.85714286 0.87894737 0.85263158 0.84736842]
[0.79166667 0.83333333 0.68085106 0.85106383 0.85106383]
[0.8148148148148148, 0.8095238095238095, 0.8421052631578947, 0.7789473684210526, 0.8]


In [90]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 20, stop = 120, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(1, 10)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, 
                               n_iter = 100, cv = 3, verbose=2, n_jobs = -1)

rf = RandomForestClassifier()
rf_random = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomizedSearchCV(estimator = rf, 
                                                        param_distributions=random_grid, 
                                                        n_iter=100, 
                                                        cv=3, 
                                                        verbose=2, 
                                                        n_jobs=-1))])
pipe_random_validator = rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:    6.7s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:   10.8s finished


In [91]:
rf_random['classifier'].best_params_

{'n_estimators': 31,
 'min_samples_split': 5,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 3,
 'bootstrap': True}

## Continue...

In [131]:
feature_importances = np.concatenate((clf.feature_importances_, 
                                      clf_2.feature_importances_,
                                      clf_3.feature_importances_,
                                      clf_4.feature_importances_), axis=1)

AxisError: axis 1 is out of bounds for array of dimension 1

In [120]:
feature_importances

array([0.03694666, 0.01348177, 0.01707419, 0.102189  , 0.07996478,
       0.07691213, 0.02267753, 0.0755486 , 0.07597982, 0.00672209,
       0.01446337, 0.13667422, 0.06993008, 0.11096635, 0.05501278,
       0.10545664])

In [126]:
# be careful with the order of columns
columns = ohe.get_feature_names().tolist() + numerical_variables.tolist() 

In [127]:
importances = pd.DataFrame(data=clf_2.feature_importances_, index = columns, columns= ['feature_importances'])

importances.sort_values(by = 'feature_importances', ascending = False)

,feature_importances
Oldpeak,0.154944
MaxHR,0.148401
ExAng,0.145547
x1_reversable,0.138458
x1_normal,0.129797
Ca,0.126637
RestBP,0.031501
Slope,0.030651
x0_nonanginal,0.028015
Age,0.022003


### Extra Material 

- [Sklearn averages probabilities in RF implementation](https://scikit-learn.org/stable/modules/ensemble.html#forest)

- [On the variance](https://newonlinecourses.science.psu.edu/stat414/node/167/)

- [Do RF immune to overfitting?](https://en.wikipedia.org/wiki/Talk%3ARandom_forest)

- [Tricky stuff with respect to feature importance](http://rnowling.github.io/machine/learning/2015/08/10/random-forest-bias.html)

- [An interesting implementation of feature importance](https://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances_faces.html#sphx-glr-auto-examples-ensemble-plot-forest-importances-faces-py)

- [Different Ensemble Methods in sklearn](https://scikit-learn.org/stable/modules/ensemble.html#forest)

- [ISLR - section 8.2](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf)

- [Another library for RF: H2o](http://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/drf.html)

H2O's version of random forest is much faster for large datasets (> million rows), handles categorical variables, built on hadoop/spark